# Análisis Exploratorio de Datos Crediticios
Proyecto en progreso – Análisis de perfiles y segmentación

## 1. Inicialización de rutas y carga de archivos

In [15]:
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

# --- Detectar carpeta base ---
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    BASE_DIR = current_dir.parent
else:
    BASE_DIR = current_dir

DATA_DIR = BASE_DIR / "data"

print(f"Carpeta base: {BASE_DIR}")
print(f"Carpeta data: {DATA_DIR}")

# --- Verificar existencia de carpeta data ---
if not DATA_DIR.exists():
    raise FileNotFoundError(f"No se encontró la carpeta data en {DATA_DIR}")

# --- Verificar archivos ---
required_files = ["personas.csv", "operaciones.csv", "household.csv"]
for file in required_files:
    file_path = DATA_DIR / file
    if file_path.exists():
        print(f"{file} encontrado")
    else:
        print(f"{file} no encontrado (continuará sin este archivo)")

# --- Configuración Pandas ---
pd.set_option('display.max_columns', None)

Carpeta base: /Users/helgazambrana/Desktop/HELGA/ESTUDIO/MKT_CS_DATOS/proyecto_crediticio
Carpeta data: /Users/helgazambrana/Desktop/HELGA/ESTUDIO/MKT_CS_DATOS/proyecto_crediticio/data
personas.csv encontrado
operaciones.csv encontrado
household.csv encontrado


## 2. Cargar datos reconociendo separador

In [17]:
path_personas = DATA_DIR / "personas.csv"
path_operaciones = DATA_DIR / "operaciones.csv"
path_household = DATA_DIR / "household.csv"

df_personas = pd.read_csv(path_personas, sep=";", low_memory=False)
df_operaciones = pd.read_csv(path_operaciones, sep=";", low_memory=False)

if path_household.exists():
    df_household = pd.read_csv(path_household, sep=";", low_memory=False)
else:
    df_household = None

print("Datos cargados")    


Datos cargados


## 3. Revisión inicial con tipos de datos

In [18]:
print("\nTipos de datos - Personas:")
print(df_personas.dtypes)

print("\nTipos de datos - Operaciones:")
print(df_operaciones.dtypes)

if df_household is not None:
    print("\nTipos de datos - Household:")
    print(df_household.dtypes)


Tipos de datos - Personas:
pers_id                 object
generaciones            object
sexo                    object
situacion_laboral       object
region_arg              object
provincia               object
nivel_bancarizacion     object
flag_prestamo          float64
flag_tarjeta             int64
ip20_continuo          float64
dtype: object

Tipos de datos - Operaciones:
pers_id                  object
ID_TRADELINE              int64
TIPO_PRODUCTO            object
PERIODO_INFO             object
FECHA_OTORGAMIENTO       object
MONT_REFE               float64
SALD_REFE               float64
SALDO_MORA              float64
CUOTA                   float64
TIPO_ENTIDAD             object
FLAG_PUBLICO_PRIVADO     object
vg_score                float64
dtype: object

Tipos de datos - Household:
periodo                  int64
pers_id_hh             float64
pers_id                 object
lider                    int64
hh_pers_cant           float64
hh_menores_cant        float64
hh_m

## 4. Validación y conversión de tipos según data dictionary

In [21]:
# Diccionario de tipos esperados (ajustar si falta alguna columna en tus CSV)
expected_types = {
    "df_personas": {
        "pers_id": str,
        "generaciones": str,
        "sexo": str,
        "situacion_laboral": str,
        "region_arg": str,
        "provincia": str,
        "nivel_bancarizacion": str,
        "flag_prestamo": "Int64",   # enteros con nulos permitidos
        "flag_tarjeta": "Int64",
        "ip20_continuo": float
    },
    "df_operaciones": {
        "pers_id": str,
        "ID_TRADELINE": str,
        "TIPO_PRODUCTO": str,
        "PERIODO_INFO": str,  # convertir a fecha más adelante
        "FECHA_OTORGAMIENTO": str,  # también a fecha después
        "MONT_REFE": float,
        "SALD_REFE": float,
        "SALDO_MORA": float,
        "CUOTA": float,
        "TIPO_ENTIDAD": str,
        "FLAG PÚBLICO PRIVADO": str,
        "vg_score": float
    },
    "df_household": {
        "periodo": str,
        "pers_id_hh": "Int64",
        "pers_id": str,
        "lider": "Int64",
        "hh_pers_cant": "Int64",
        "hh_menores_cant": "Int64",
        "hh_mayores_cant": "Int64",
        "hh_adulmayores_cant": "Int64",
        "hh_menores_prop": float,
        "hh_bebe_flag": "Int64",
        "hh_tipo_familia": str,
        "hh_ip_lider": float,
        "hh_ip_lider_cat": str,
        "hh_ip_percapita": float,
        "hh_score_lider": float,
        "hh_score_prom": float
    }
}

# Función para convertir tipos
def enforce_dtypes(df, dtypes_map, df_name):
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                df[col] = df[col].astype(dtype)
            except Exception as e:
                print(f"No se pudo convertir {df_name}.{col} a {dtype}: {e}")
    return df

# Aplicar conversión
df_personas = enforce_dtypes(df_personas, expected_types["df_personas"], "df_personas")
df_operaciones = enforce_dtypes(df_operaciones, expected_types["df_operaciones"], "df_operaciones")

if df_household is not None:
    df_household = enforce_dtypes(df_household, expected_types["df_household"], "df_household")

print("Conversión de tipos finalizada")


Conversión de tipos finalizada


## 5. Resumen post-transformación

In [22]:
print("\nResumen de tipos después de conversión - Personas:")
print(df_personas.dtypes)

print("\nResumen de tipos después de conversión - Operaciones:")
print(df_operaciones.dtypes)

if df_household is not None:
    print("\nResumen de tipos después de conversión - Household:")
    print(df_household.dtypes)


Resumen de tipos después de conversión - Personas:
pers_id                 object
generaciones            object
sexo                    object
situacion_laboral       object
region_arg              object
provincia               object
nivel_bancarizacion     object
flag_prestamo            Int64
flag_tarjeta             Int64
ip20_continuo          float64
dtype: object

Resumen de tipos después de conversión - Operaciones:
pers_id                  object
ID_TRADELINE             object
TIPO_PRODUCTO            object
PERIODO_INFO             object
FECHA_OTORGAMIENTO       object
MONT_REFE               float64
SALD_REFE               float64
SALDO_MORA              float64
CUOTA                   float64
TIPO_ENTIDAD             object
FLAG_PUBLICO_PRIVADO     object
vg_score                float64
dtype: object

Resumen de tipos después de conversión - Household:
periodo                 object
pers_id_hh               Int64
pers_id                 object
lider                    